# Objectives

Clean up the natural gas billing data from Client 2 and output the prorated consumption and charge values by account and calendar month.

- Input: raw data
- Outupt: 
    - df_cleaned
    - df_prorated
    - df_mapping
    - prorated_ts
    - prorated_ts.csv

# Step 1: Load packages and Data

In [1]:
import pandas as pd
import numpy as np
import pandasql as pdsql
import math

from datetime import timedelta, datetime
from dateutil.relativedelta import *

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
# initiate notebook for offline plot
init_notebook_mode(connected=True)         

### Specify the output directory of the resulting datasets

In [2]:
output_directory = '../output/client2/natural_gas/'

In [3]:
file_name = "/Users/feiwang/Desktop/capstone/anomalyDetectors/data/Client 2 - Data for UW team.xlsx"
xl_file = pd.ExcelFile(file_name)

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

df_orig = dfs['Sheet1']

### Add a row_id in the original dataset for debugging

In [4]:
df_orig = df_orig.reset_index()
df_orig.rename(columns = {'index':'row_id'}, inplace = True)

### Create a data frame to log the rows with data quality issues.

In [5]:
df_flags = pd.DataFrame(columns = ['row_id', 'flag'])

### Create a working dataframe and only focus on electricity bills

In [6]:
df = df_orig.copy()

In [7]:
df.rename(columns = {'Building ID':'Building_ID', 'Meter_ID':'Meter_Number', 'Cost':'Charge', \
                    'Start Date':'Service_Start_Date', 'End Date':'Service_End_Date', 'Units.1':'Units'},\
          inplace = True)

In [10]:
mask = df.Fuel == 'Natural Gas'
df = df[mask]

# Step 2: Data Cleaning & Exploration

## Part I: Cleaning (necessary data cleaning work before prorating of consumption and charge values to calendar months)

### 1. Remove the two columns on demand and its units since we only care about consumption & charges

In [11]:
df.head()

,row_id,Building_ID,Fuel,Meter_Number,Account_Name,Service_Start_Date,Service_End_Date,Consumption,Units,Charge,Currency,Demand,Units,Vendor,Invoice Number
70,70,47068,Natural Gas,MN10002,AN102,2015-12-09,2016-01-07,1750.0,therms,1137.62,USD,0.0,kW,V1,IN51
71,71,47068,Natural Gas,MN10002,AN102,2016-01-08,2016-02-05,927.9,therms,662.08,USD,0.0,kW,V1,IN52
72,72,47068,Natural Gas,MN10002,AN102,2016-02-06,2016-03-08,413.0,therms,327.38,USD,0.0,kW,V1,IN53
73,73,47068,Natural Gas,MN10002,AN102,2016-03-09,2016-04-06,1151.0,therms,779.35,USD,0.0,kW,V1,IN54
74,74,47068,Natural Gas,MN10002,AN102,2016-04-07,2016-05-06,694.0,therms,423.47,USD,0.0,kW,V1,IN55


In [12]:
df = pd.concat([df.iloc[:, 0:11], df.iloc[:, 13:15]], axis = 1)

### 2. Remove rows with missing values in Building_ID, Meter_Number, Service_Start_Date, Service_End_Date, Consumption or Charge (we need these columns to do value proration)

In [13]:
df.isna().sum()

row_id                0
Building_ID           0
Fuel                  0
Meter_Number          0
Account_Name          0
Service_Start_Date    0
Service_End_Date      0
Consumption           0
Units                 0
Charge                0
Currency              0
Vendor                0
Invoice Number        0
dtype: int64

### 3. Ensure correct data types for the six columns

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1637 entries, 70 to 4588
Data columns (total 13 columns):
row_id                1637 non-null int64
Building_ID           1637 non-null int64
Fuel                  1637 non-null object
Meter_Number          1637 non-null object
Account_Name          1637 non-null object
Service_Start_Date    1637 non-null datetime64[ns]
Service_End_Date      1637 non-null datetime64[ns]
Consumption           1637 non-null float64
Units                 1637 non-null object
Charge                1637 non-null float64
Currency              1637 non-null object
Vendor                1637 non-null object
Invoice Number        1637 non-null object
dtypes: datetime64[ns](2), float64(2), int64(2), object(7)
memory usage: 179.0+ KB


### 4. Ensure each Meter_number is mapped to only one Building_ID

In [15]:
tmp = df.groupby(['Meter_Number']).agg({'Building_ID':'nunique'}).reset_index()

In [16]:
tmp[tmp['Building_ID'] != 1]

,Meter_Number,Building_ID
42,MN10070,2


In [21]:
mask = df.Meter_Number.isin(tmp[tmp['Building_ID'] != 1].Meter_Number.values)

df_flags = pd.concat([df_flags, pd.DataFrame({'row_id':df.loc[mask, ].row_id.values, 'flag':'Meter_Number corresponds to multipel building_id, rows removed'})])

df = df[~mask]
df = df.reset_index(drop=True)

### 5. Check if there needs Meter_Number merging under the same Building_ID 

In [22]:
tmp = df.groupby(['Building_ID', 'Meter_Number']).agg({'Service_Start_Date':'min', 'Service_End_Date':'max'}).reset_index()

tmp.rename(columns = {'Service_Start_Date':'Min_Start_Date', 'Service_End_Date':'Max_End_Date'}, inplace = True)

tmp.loc[:, 'Service_Duration'] = tmp.apply(lambda x: (x['Max_End_Date'] - x['Min_Start_Date']).days, axis = 1)

df_duration = tmp.copy()
del(tmp)

In [23]:
tmp = pd.merge(df_duration, df_duration, on = 'Building_ID', how = 'inner')\
    [['Building_ID', 'Meter_Number_x', 'Meter_Number_y', 'Min_Start_Date_x', 'Max_End_Date_x', 'Min_Start_Date_y', 'Max_End_Date_y']]

mask = tmp['Meter_Number_x'] != tmp['Meter_Number_y']
tmp = tmp[mask]

In [24]:
tmp.loc[:, 'date_diff'] = tmp.apply(lambda x: (x['Min_Start_Date_x'] - x['Max_End_Date_y']).days, axis = 1)

In [26]:
mask = (tmp.date_diff >= -1) & (tmp.date_diff <= 1)
df_meter_mapping = tmp[mask]

### Find the one-to-one Meter_Number mappings, merge them in the df dataset

In [36]:
tmp = df_meter_mapping.groupby('Meter_Number_x').count().reset_index().iloc[:, 0:2]
tmp = tmp.rename(columns = {'Building_ID':'Count'})
mask = df_meter_mapping.Meter_Number_x.isin(tmp[tmp.Count == 1].Meter_Number_x.values)

In [52]:
df = df.merge(df_meter_mapping[mask][['Building_ID', 'Meter_Number_x', 'Meter_Number_y']]\
         , left_on = ['Building_ID', 'Meter_Number'], right_on = ['Building_ID', 'Meter_Number_x']\
         , how = 'left')

df.loc[~df['Meter_Number_y'].isnull(), 'Meter_Number'] = df.loc[~df['Meter_Number_y'].isnull(), 'Meter_Number_y'] 

df.drop(['Meter_Number_x', 'Meter_Number_y'], axis = 1, inplace = True)

### For the many-to-many Meter_Number mappings, manually check them one-by-one and do the merging

In [57]:
df_meter_mapping[~mask]

,Building_ID,Meter_Number_x,Meter_Number_y,Min_Start_Date_x,Max_End_Date_x,Min_Start_Date_y,Max_End_Date_y,date_diff
225,47092,MN10089,MN10086,2017-01-26,2018-12-24,2016-01-27,2017-01-25,1
226,47092,MN10089,MN10087,2017-01-26,2018-12-24,2015-12-25,2017-01-25,1
245,47093,MN10098,MN10096,2017-01-26,2018-12-24,2015-12-25,2017-01-25,1
246,47093,MN10098,MN10097,2017-01-26,2018-12-24,2016-04-03,2017-01-25,1
250,47093,MN10099,MN10096,2017-01-26,2018-12-24,2015-12-25,2017-01-25,1
251,47093,MN10099,MN10097,2017-01-26,2018-12-24,2016-04-03,2017-01-25,1
317,47094,MN10111,MN10108,2017-01-26,2018-12-24,2015-12-25,2017-01-25,1
319,47094,MN10111,MN10110,2017-01-26,2018-12-24,2016-01-28,2017-01-25,1
327,47094,MN10112,MN10108,2017-01-26,2018-12-24,2015-12-25,2017-01-25,1
329,47094,MN10112,MN10110,2017-01-26,2018-12-24,2016-01-28,2017-01-25,1


In [58]:
mask = (df.Building_ID == 47093) & (df.Meter_Number == 'MN10098')
df.loc[mask, 'Meter_Number'] = 'MN10096'

In [59]:
mask = (df.Building_ID == 47093) & (df.Meter_Number == 'MN10099')
df.loc[mask, 'Meter_Number'] = 'MN10097'

In [60]:
mask = (df.Building_ID == 47095) & (df.Meter_Number == 'MN10123')
df.loc[mask, 'Meter_Number'] = 'MN10121'

In [61]:
mask = (df.Building_ID == 47095) & (df.Meter_Number == 'MN10124')
df.loc[mask, 'Meter_Number'] = 'MN10122'

### 6. Ensure the service dates columns are correct

In [62]:
mask = df['Service_Start_Date'] >= df['Service_End_Date']
df_flags = pd.concat([df_flags, pd.DataFrame({'row_id':df.loc[mask, ].row_id.values, 'flag':'incorrect service dates, rows removed'})])
df = df[~mask]
df = df.reset_index(drop=True)

### 7. Assign an Account ID for each row

In [63]:
df.loc[:, 'Account'] = df.apply(lambda x: str(x['Building_ID']) + '-' + str(x['Meter_Number']), axis = 1)

## Part II: Exploration (potential data issues that do not prohibit data proration, and which we hope can be detected automatically later via our anomalty detection rules)

### 1. Check the currency column

In [64]:
df.Currency.value_counts()

USD    1610
Name: Currency, dtype: int64

### 2. Check the Unit column

#### There is no unit value with typo or caption variation

In [65]:
df.Units.value_counts()

therms    1208
Dth        249
ccf         46
MMBtu       46
m3          39
each        22
Name: Units, dtype: int64

### 3. Check rows that have 0 in either consumption or cost column

- 33% of the rows have either 0 value in consumption or cost column
- Shall we remove them? No
    - sometimes there is legitimate reasons (the consumption is below a certain threshold and thus there was no charge)
    - hope our anomaly detection rules can find them without our manual effort of data cleaning

In [66]:
mask1 = df.Consumption == 0
mask2 = df.Charge == 0

In [67]:
df[mask1 & mask2].shape[0]/df.shape[0]

0.0055900621118012426

In [68]:
df[mask1 & ~mask2].shape[0]/df.shape[0]

0.3279503105590062

In [69]:
df[~mask1 & mask2].shape[0]/df.shape[0]

0.0

In [70]:
df[~mask1 & ~mask2].shape[0]/df.shape[0]

0.6664596273291925

### Check the overlapping of service periods

In [71]:
# order by dataframe and assign a row number
df = df.sort_values(by = ['Building_ID', 'Meter_Number', 'Service_Start_Date', 'Service_End_Date'], ascending=[True, True, True, True]).copy()
df = df.reset_index(drop=True)
df = df.reset_index()
df.rename(columns = {'index':'row_number'}, inplace = True)

prev = df.loc[:, ['row_number', 'Building_ID', 'Meter_Number', 'Service_Start_Date', 'Service_End_Date', 'Consumption', 'Charge']]

prev.loc[:, 'row_number'] = prev['row_number'].map(lambda x: x + 1)


tmp = df[['row_number', 'row_id', 'Building_ID', 'Meter_Number', 'Service_Start_Date', 'Service_End_Date', 'Consumption', 'Charge']]\
.merge(prev, on = ['row_number', 'Building_ID', 'Meter_Number'], how = 'left')

tmp.columns = ['row_number', 'row_id', 'Building_ID', 'Meter_Number', 'Service_Start_Date',
       'Service_End_Date', 'Consumption', 'Current_Charges', 'Service_Start_Date_Prev', 'Service_End_Date_Prev', 'Consumption_Prev', 'Current_Charges_Prev']

tmp.loc[:, 'Diff_Prev'] = tmp.apply(lambda x: (x['Service_Start_Date'] - x['Service_End_Date_Prev']).days, axis = 1)

In [72]:
rows = list(tmp[tmp.Diff_Prev < -2].row_number.values)

### We have some bills with overlapping billing periods (2.9%)

In [73]:
len(rows)/df.shape[0]

0.029192546583850933

### Drop the row_number field after we finish matching the adjacent rows

In [74]:
df.drop('row_number', axis = 1, inplace = True)

### Save a copy of the cleaned version of the dataset

In [75]:
df_cleaned = df.copy()

# Step 4: Prorate the bills to calendar months

### Check if the service_end_date is inclusive for each bill

In [76]:
# order by dataframe and assign a row number
df = df.sort_values(by = ['Building_ID', 'Meter_Number', 'Service_Start_Date', 'Service_End_Date'], ascending=[True, True, True, True]).copy()
df = df.reset_index(drop=True)
df = df.reset_index()
df.rename(columns = {'index':'row_number'}, inplace = True)

In [77]:
cols = ['row_number', 'row_id', 'Account', \
        'Service_Start_Date', 'Service_End_Date', 'Consumption', 'Charge']
df = df[cols]

In [78]:
df_next = df.loc[:, ['row_number', 'Account', 'Service_Start_Date', 'Service_End_Date']]
df_next.loc[:, 'row_number'] = df_next.row_number.map(lambda x: x - 1)

tmp = df.merge(df_next, on = ['row_number', 'Account'], how = 'left')

In [79]:
tmp.columns = ['row_number', 'row_id', 'Account', 
       'Service_Start_Date', 'Service_End_Date', 'Consumption', 'Charge',
       'Service_Start_Date_Next', 'Service_End_Date_Next']

tmp.loc[:, 'Diff_Next'] = tmp.apply(lambda x: (x['Service_Start_Date_Next'] - x['Service_End_Date']).days, axis = 1)

In [80]:
df = tmp.copy()
del(tmp)

### Most cases, the start_date of the next bill is 1 day after the end_date of the previous bill (end_date is inclusive)

In [81]:
df.Diff_Next.value_counts().sort_index()

-89.0       1
-58.0       9
-30.0      10
-29.0      16
-28.0       8
-27.0       2
-6.0        1
 0.0       15
 1.0     1467
 9.0        1
 31.0       1
 59.0       1
Name: Diff_Next, dtype: int64

### Add the month associated with service_start_date and service_end_date

In [82]:
df['Start_Date_Month'] = df['Service_Start_Date'].apply(\
  lambda x: pd.to_datetime('-'.join([str(x.year), str(x.month)])))

df['End_Date_Month'] = df.apply(\
  lambda x: pd.to_datetime('-'.join([str(x['Service_End_Date'].year), str(x['Service_End_Date'].month)]))\
            if x['Diff_Next'] == 1 else pd.to_datetime('-'.join([str((x['Service_End_Date'] + relativedelta(days=-1)).year), str((x['Service_End_Date'] + relativedelta(days=-1)).month)]))\
            ,axis = 1)

### Create a dataframe of the relevant columns to work on the mapping between row_id to the calendar month

In [83]:
cols = ['row_id', 'Start_Date_Month', 'End_Date_Month']
temp = df[cols]

Create a new data frame to store the mapping. The dataframe will have 3 columns: 'row_id' (identifier of the bill), 'Start_Date_Month' and 'End_Date_Month'. We'll collapse the last 2 columns into 1 in order to get the associated calendar month for each bill.

First, save all the row-month mapping between row_id and its start_date_month and end_date_month to the new dataframe

In [84]:
df_month_row_mapping = temp.copy()

Second, there are cases where the billing window is longer than one calendar month. 

- So for each bill, check if the billing window is longer than one month;
If so, save the Start_Date_Month in df_month_row_mapping and then replace it with its subsequent month until the billing window is less than one month.

In [85]:
while (temp.shape[0] > 0):
    temp.loc[:, 'Start_Date_Month_Next'] = \
    temp['Start_Date_Month'].map(lambda x: x + relativedelta(months=+1))

    temp.loc[:, 'Ind'] = \
    temp.apply(lambda x: 1 if x['Start_Date_Month_Next'] < x['End_Date_Month'] else 0, axis = 1)


    mask = temp['Ind'] == 1
    temp = temp.loc[mask,['row_id', 'Start_Date_Month_Next', 'End_Date_Month']].copy()
    temp.columns = ['row_id', 'Start_Date_Month', 'End_Date_Month']

    df_month_row_mapping = pd.concat([df_month_row_mapping, temp])

/Users/feiwang/miniconda3/envs/uwcapstone/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/feiwang/miniconda3/envs/uwcapstone/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Collapse the  'Start_Date_Month' and 'End_Date_Month' columns into one column that contains all corresponding calendar months of a given bill.

In [86]:
temp = pd.melt(df_month_row_mapping, id_vars = df_month_row_mapping.columns[0:-2].values, value_vars = df_month_row_mapping[cols].columns[-2:])
temp.drop('variable', axis = 1, inplace = True)

temp = temp.drop_duplicates()
temp.columns  = ['row_id', 'Month']

Add calendar month to each bill.

In [87]:
temp = pd.merge(temp, df, on = 'row_id', how = 'left')

For each bill, calculate the number of days prorated into the corresponding calendar month.

In [88]:
temp.loc[:, 'Prorated_Days'] = \
temp.apply(lambda x: \
       (min(x['Month'] + relativedelta(months = 1), (x['Service_End_Date'] + relativedelta(days=1))) \
        - max(x['Service_Start_Date'], x['Month'])).days if x['Diff_Next'] == 1\
       else (min(x['Month'] + relativedelta(months = 1), x['Service_End_Date']) \
        - max(x['Service_Start_Date'], x['Month'])).days\
       , axis = 1) 

For each bill, calculate the number of days covered by the bill.

In [89]:
temp.loc[:, 'Bill_Duration'] = \
temp.apply(lambda x: (x['Service_End_Date'] + relativedelta(days=1) - x['Service_Start_Date']).days\
       if x['Diff_Next'] == 1\
       else (x['Service_End_Date'] - x['Service_Start_Date']).days\
       , axis = 1) 

Calculate the prorated consumption and charge values based on the prorated days and bill duration.

In [90]:
temp.loc[:, 'Prorated_Consumption'] = \
temp.apply(lambda x: (x['Consumption'] / x['Bill_Duration'] )* x['Prorated_Days'], axis = 1)

In [91]:
temp.loc[:, 'Prorated_Charge'] = \
temp.apply(lambda x: (x['Charge'] / x['Bill_Duration'] )* x['Prorated_Days'], axis = 1)

Save a dataframe that contains the full dataset as well as the corresponding calendar month for each bill. This dataframe will be useful to calculate the prorated consumption and charge values per account-month. Note this dataframe has more rows than the original dataset since one bill may correspond to multiple calendar months.

In [92]:
df_with_calendar_month = temp.copy()

Aggregate the data to Account-Month level by summing up the prorated kwh consupmtion values per calendar_month.

In [93]:
df_prorated = \
df_with_calendar_month.groupby(['Account','Month']).\
    agg({'Prorated_Consumption':'sum', 'Prorated_Charge':'sum', 'Prorated_Days':'sum'}).reset_index()

### Create a mapping table between the individual bill and account_month instance

In [94]:
df_prorated = df_prorated.reset_index()

# add row_id to the df_prorated dataset
df_prorated.rename(columns = {'index':'row_id'}, inplace = True)

df_mapping = pd.merge(df_with_calendar_month, df_prorated, on = ['Account', 'Month'], how = 'inner')[['row_id_x', 'row_id_y']]
df_mapping.rename(columns = {'row_id_x':'row_id_account_month', 'row_id_y':'row_id_bill'}, inplace = True)

So far for each account we've only been working on the calendar months that the accounts has billing records. We also need to map the account id to the calendar months where it should have data but were not logged or reported.

### Create a dataframe that maps the account with all the calendar months that it should have bills

Find all unique accounts (Building_Meter) and months in the dataset.

In [95]:
accounts = df_with_calendar_month.Account.value_counts().index.values

end = df_with_calendar_month['Month'].max()
start = df_with_calendar_month['Month'].min()
diff = (end.year - start.year) * 12 + end.month - start.month
# list of unique months
months = [start + relativedelta(months=x) for x in range(0, diff + 1)]

Create a reference table with all the calendar months and the corresponding # of days in the month. 

In [96]:
month_days = [(x + relativedelta(months = 1) - x).days for x in months]
df_month_days = pd.DataFrame({'Month':months,  'Month_#_Days':month_days})

Now we can map the account (Building_Meter) to all the calendar months that it should have billing data (Here we assumed the account should have data in all months between the first and last calendar month that it has billing data of).

In [97]:
df_account_month = pd.DataFrame()

for j in range(len(accounts)):
    mask = (df_with_calendar_month['Account'] == accounts[j])
    start = df_with_calendar_month[mask]['Month'].min()
    end = df_with_calendar_month[mask]['Month'].max()
    start_index = months.index(start)
    end_index = months.index(end)
    
    temp_df = pd.DataFrame({'Account':accounts[j], 'Month':months[start_index:end_index + 1]})
    temp_df.loc[:, 'Month_Type'] = 'Month_In_The_Middle'
    temp_df.loc[0, 'Month_Type'] = 'First_Month'
    temp_df.loc[temp_df.shape[0]-1, 'Month_Type'] = 'Last_Month'
    df_account_month = pd.concat([df_account_month, temp_df])

In [98]:
df_account_month = pd.merge(df_account_month, df_month_days, on = ['Month'], how = 'left')

Left join account_meter mapping table to get all months for each account.

In [99]:
df_prorated = pd.merge(df_account_month, df_prorated, on = ['Account', 'Month'], how = 'left')

For months that the account didn't have data, fill in with NA's.

In [100]:
mask = df_prorated['Prorated_Days'].isnull()
df_prorated.loc[mask, 'Prorated_Consumption'] = np.nan
df_prorated.loc[mask, 'Prorated_Charge'] = np.nan
df_prorated.loc[mask, 'NA_Ind'] = True

## Step 5: Save the datasets

In [102]:
df_mapping.to_pickle(output_directory + 'df_mapping')
df_prorated.to_pickle(output_directory + 'df_prorated')
df_cleaned.to_pickle(output_directory + 'df_cleaned')
df_orig.to_pickle(output_directory + 'df_orig')

#### Find the accounts that have no month with missing data and at least 2 years of monthly data for anomaly analysis.

In [132]:
tmp = pd.merge(
    df_prorated.groupby(['Account']).agg({'Month':'count'}).reset_index()\
    , df_prorated.groupby(['Account']).agg({'NA_Ind':'sum'}).reset_index()\
    , on = 'Account', how = 'inner')

tmp.rename(columns = {'Month':'Count', 'NA_Ind':'NA_Count'}, inplace = True)
mask = (tmp['NA_Count'] == 0) & (tmp['Count'] >= 24)

In [133]:
mask = df_prorated.Account.isin(tmp[mask].Account.values)
df_prorated[mask][['Account', 'Month', 'Prorated_Consumption', 'Prorated_Charge']].to_pickle(output_directory + 'natural_gas_prorated_ts')
df_prorated[mask][['Account', 'Month', 'Prorated_Consumption', 'Prorated_Charge']].to_csv(output_directory + 'natural_gas_prorated_ts.csv')